In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report
%matplotlib inline

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
data=pd.read_csv("/content/drive/MyDrive/Dataset_SRIP/UNSW_2018_IoT_Botnet_Full5pc_1.csv")
data=pd.concat([data,pd.read_csv(r"/content/drive/MyDrive/Dataset_SRIP/UNSW_2018_IoT_Botnet_Full5pc_2.csv")])
data=pd.concat([data,pd.read_csv(r"/content/drive/MyDrive/Dataset_SRIP/UNSW_2018_IoT_Botnet_Full5pc_3.csv")])
data=pd.concat([data,pd.read_csv(r"/content/drive/MyDrive/Dataset_SRIP/UNSW_2018_IoT_Botnet_Full5pc_4.csv")])

<ipython-input-30-58268d3b2303>:1: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("/content/drive/MyDrive/Dataset_SRIP/UNSW_2018_IoT_Botnet_Full5pc_1.csv")
<ipython-input-30-58268d3b2303>:2: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.concat([data,pd.read_csv(r"/content/drive/MyDrive/Dataset_SRIP/UNSW_2018_IoT_Botnet_Full5pc_2.csv")])
<ipython-input-30-58268d3b2303>:3: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.concat([data,pd.read_csv(r"/content/drive/MyDrive/Dataset_SRIP/UNSW_2018_IoT_Botnet_Full5pc_3.csv")])
<ipython-input-30-58268d3b2303>:4: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.concat([data,pd.read_csv(r"/content/drive/MyDrive/Dataset_SRIP/UNSW_2018_IoT_Botnet_Full5pc_4.csv")])


In [31]:
data.head()

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,1,1.528089e+09,e,1,tcp,1,192.168.100.147,49960,192.168.100.7,80,...,1.12704,96,75,1.133720,1.129970,770,602,1,DoS,HTTP
1,2,1.528089e+09,e,1,arp,2,192.168.100.7,-1,192.168.100.147,-1,...,15267.20000,1,2,0.005142,0.005142,2,6,1,DoS,HTTP
2,3,1.528089e+09,e,1,tcp,1,192.168.100.147,49962,192.168.100.7,80,...,1.12704,96,75,1.135100,1.129970,770,602,1,DoS,HTTP
3,4,1.528089e+09,e,1,tcp,1,192.168.100.147,49964,192.168.100.7,80,...,1.12704,96,75,1.135140,1.129970,770,602,1,DoS,HTTP
4,5,1.528089e+09,e,1,tcp,1,192.168.100.147,49966,192.168.100.7,80,...,1.12704,96,75,1.135260,1.129970,770,602,1,DoS,HTTP


In [32]:
data.columns

Index(['pkSeqID', 'stime', 'flgs', 'flgs_number', 'proto', 'proto_number',
       'saddr', 'sport', 'daddr', 'dport', 'pkts', 'bytes', 'state',
       'state_number', 'ltime', 'seq', 'dur', 'mean', 'stddev', 'sum', 'min',
       'max', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate',
       'TnBPSrcIP', 'TnBPDstIP', 'TnP_PSrcIP', 'TnP_PDstIP', 'TnP_PerProto',
       'TnP_Per_Dport', 'AR_P_Proto_P_SrcIP', 'AR_P_Proto_P_DstIP',
       'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP', 'AR_P_Proto_P_Sport',
       'AR_P_Proto_P_Dport', 'Pkts_P_State_P_Protocol_P_DestIP',
       'Pkts_P_State_P_Protocol_P_SrcIP', 'attack', 'category', 'subcategory'],
      dtype='object')

In [33]:
data=data.drop(columns=["flgs","proto","state"])

In [34]:
data['category'].value_counts()

,count
category,
DDoS,1926624
DoS,1650260
Reconnaissance,91082
Normal,477
Theft,79


In [35]:
data.describe()

,pkSeqID,stime,flgs_number,proto_number,pkts,bytes,state_number,ltime,seq,dur,...,TnP_Per_Dport,AR_P_Proto_P_SrcIP,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack
count,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,...,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06,3.668522e+06
mean,1.834262e+06,1.528050e+09,1.490206e+00,2.095884e+00,7.725963e+00,8.690501e+02,3.134390e+00,1.528050e+09,1.213204e+05,2.033479e+01,...,7.369070e+02,3.327439e+02,2.851832e+02,9.245168e+01,8.253848e+01,4.564945e+02,5.385196e+02,6.422897e+02,5.859984e+02,9.998700e-01
std,1.059011e+06,2.581498e+05,8.652444e-01,9.990837e-01,1.155876e+02,1.122667e+05,1.186971e+00,2.581507e+05,7.579428e+04,2.148764e+01,...,6.527134e+02,8.466031e+03,4.096943e+03,1.817643e+01,2.439739e+01,1.432917e+04,1.569824e+04,4.533432e+02,4.332619e+02,1.140212e-02
min,1.000000e+00,1.526344e+09,1.000000e+00,1.000000e+00,1.000000e+00,6.000000e+01,1.000000e+00,1.526344e+09,1.000000e+00,0.000000e+00,...,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,9.171312e+05,1.528085e+09,1.000000e+00,1.000000e+00,5.000000e+00,4.200000e+02,3.000000e+00,1.528085e+09,5.487600e+04,1.256256e+01,...,5.000000e+02,2.359950e-01,2.436680e-01,1.000000e+02,6.900000e+01,2.314810e-01,2.457730e-01,3.240000e+02,2.940000e+02,1.000000e+00
50%,1.834262e+06,1.528096e+09,1.000000e+00,3.000000e+00,7.000000e+00,6.000000e+02,4.000000e+00,1.528096e+09,1.177690e+05,1.550852e+01,...,7.000000e+02,3.900890e-01,3.986290e-01,1.000000e+02,1.000000e+02,3.785910e-01,3.943060e-01,6.000000e+02,5.000000e+02,1.000000e+00
75%,2.751392e+06,1.528099e+09,2.000000e+00,3.000000e+00,9.000000e+00,7.700000e+02,4.000000e+00,1.528099e+09,1.849300e+05,2.709986e+01,...,9.200000e+02,5.725580e-01,5.796390e-01,1.000000e+02,1.000000e+02,5.725550e-01,5.769710e-01,8.280000e+02,8.000000e+02,1.000000e+00
max,3.668522e+06,1.529381e+09,9.000000e+00,5.000000e+00,7.005700e+04,7.183334e+07,1.100000e+01,1.529382e+09,2.622120e+05,2.771485e+03,...,2.444250e+05,2.714290e+06,1.000000e+06,1.000000e+02,1.000000e+02,3.000000e+06,2.000000e+06,1.125440e+05,1.179390e+05,1.000000e+00


In [36]:
data["proto"].value_counts()

KeyError: 'proto'

In [37]:
data["saddr"].value_counts()

,count
saddr,
192.168.100.147,950966
192.168.100.148,923290
192.168.100.150,890262
192.168.100.149,890146
192.168.100.3,8281
192.168.100.5,5118
192.168.100.6,327
192.168.100.7,43
192.168.100.4,19


In [38]:
data["sport"].value_counts()

,count
sport,
0x0303,8950
80,3986
2290,1041
2456,1039
1822,1037
...,...
51305,1
51313,1
51311,1


In [39]:
data["daddr"].value_counts()

,count
daddr,
192.168.100.3,2375733
192.168.100.5,451694
192.168.100.7,414859
192.168.100.6,412209
192.168.100.150,3860
...,...
205.251.198.119,1
205.251.195.185,1
192.48.79.30,1


In [40]:
data["dport"].value_counts()

,count
dport,
80,3225356
80,348219
1,6745
3306,4613
53,340
...,...
30559,1
30558,1
30556,1


In [41]:
data["attack"].value_counts()

,count
attack,
1,3668045
0,477


In [42]:
data["subcategory"].value_counts()

,count
subcategory,
UDP,1981230
TCP,1593180
Service_Scan,73168
OS_Fingerprint,17914
HTTP,2474
Normal,477
Keylogging,73
Data_Exfiltration,6


In [43]:
target_features = data[['attack','category','subcategory']]
features= data.drop(['attack','category','subcategory'], axis=1)

In [44]:
le = LabelEncoder()
target_features['category'] = le.fit_transform(target_features['category'])
target_features['subcategory'] = le.fit_transform(target_features['subcategory'])

target_features.head()

<ipython-input-44-1ca8795a88ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_features['category'] = le.fit_transform(target_features['category'])
<ipython-input-44-1ca8795a88ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_features['subcategory'] = le.fit_transform(target_features['subcategory'])


,attack,category,subcategory
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1


In [45]:
X_train, X_test, y_train, y_test = train_test_split(features,target_features)

In [46]:
# Scaling the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

ValueError: could not convert string to float: '192.168.100.147'